In [5]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from spotipy.oauth2 import SpotifyOAuth


cid = '5d9494efb79d4abaabc13e9778431f49'
secret = '4d23d535c7fe4e9cadd241ddf58df15e'
client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [4]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0, 100):
    track_results = sp.search(q='year:2018', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [14]:
# for i in range(0, 100):
#     track_results = sp.search(q='year:2018', type='track', limit=50, offset=i)
#     for i, t in enumerate(track_results['tracks']['items']):
#         artist_name.append(t['artists'][0]['name'])
#         track_name.append(t['name'])
#         track_id.append(t['id'])
#         popularity.append(t['popularity'])

for i in range(0, 100):
    track_results = sp.search(q='year:2018', type='track', limit=50, offset=i)
    [
        t['artists'][0]['name']
        for i, t in enumerate(track_results['tracks']['items'])
    ]

[
    [t['artists'][0]['name']
        for i, t in enumerate(track_results['tracks']['items'])]
]


ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out.

In [ ]:
track_name


['Whiskey Glasses',
 'I Was Never There',
 'SLOW DANCING IN THE DARK',
 'Call Out My Name',
 'Drip Too Hard (Lil Baby & Gunna)',
 'Yes Indeed',
 "Chasin' You",
 'All Girls Are The Same',
 'Lucid Dreams',
 'ball w/o you',
 'Softcore',
 'lovely (with Khalid)',
 'Beautiful Crazy',
 'Pure Cocaine',
 'Hope',
 'Shoota (feat. Lil Uzi Vert)',
 'a lot',
 'Lean Wit Me',
 'rockstar (feat. 21 Savage)',
 'Telephones',
 'Violent Crimes',
 'Washing Machine Heart',
 'You Get Me So High',
 'All The Stars (with SZA)',
 'Self Care',
 'SAD!',
 'Taking A Walk',
 'Space Cadet (feat. Gunna)',
 'Sunflower - Spider-Man: Into the Spider-Verse',
 'Armed And Dangerous',
 "God's Plan",
 'Moonlight',
 'Going Bad (feat. Drake)',
 'Ghost Town',
 'She Got the Best of Me',
 '1400 / 999 Freestyle',
 '10 Freaky Girls (with 21 Savage)',
 'After The Storm (feat. Tyler, The Creator & Bootsy Collins)',
 'Little Dark Age',
 'SICKO MODE',
 'One Kiss (with Dua Lipa)',
 'Show Me How',
 'Prom Queen',
 'Better Now',
 'YEAH RIGHT',

In [6]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])


Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)
